In [ ]:
!pip install google-cloud-automl
!pip install google-cloud-storage
!pip install opencv-python==3.3.0.9
!pip install keras
!pip install https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl

In [1]:
from google.datalab import storage
from google.cloud import storage
from IPython.display import Image
from PIL import Image as Img
from imageai.Detection import ObjectDetection
import math
import cv2
import numpy as np
import io
import glob

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = 'saleshousephotos'

bucket = storage_client.get_bucket(bucket_name)
blobs = bucket.list_blobs()

photos = []
for blob in blobs:
  if 'photos' in blob.name:
    photos.append(blob.name)
photos = photos[1:]

def blur_image(image, amount=5):
    '''Blurs the image
    Does not affect the original image'''
    kernel = np.ones((amount, amount), np.float32) / (amount**2)
    return cv2.filter2D(image, -1, kernel)

In [3]:
#for i in range(len(photos)):
for i in range(1):
  tmp = bucket.blob(photos[i]).download_as_string()
  img = Img.open(io.BytesIO(tmp))
  img = np.array(img)
    
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  edges = cv2.Canny(gray, 250, 250)
  image = blur_image(edges, amount = 7)

  lines = cv2.HoughLinesP(image, rho = 1, theta = math.pi, threshold = 20, minLineLength = img.shape[0]/1.5, maxLineGap = 0)

  if lines is None:
      lines = cv2.HoughLinesP(image, rho = 1, theta = math.pi, threshold = 50, minLineLength = img.shape[0]/2.5, maxLineGap = 3)

  img2 = img.copy()
  lines = sorted(lines,key=lambda x: x[0,0])
  dominance = -1000
  right_chop = img.shape[1]
  left_chop = 0

  for line2 in range(len(lines)):
      line = lines[line2]
      if line[0,0] - dominance > 0.2*img.shape[1]:
          if (line[0,0] < 0.2*img.shape[1] or line[0,0] > 0.8*img.shape[1]):
              if (line[0,0] < 0.2*img.shape[1]):
                left_chop = line[0,0]
              if (line[0,0] > 0.8*img.shape[1]):
                right_chop = line[0,0]
              pt1 = (line[0,0],line[0,1])
              pt2 = (line[0,2],line[0,3])
              cv2.line(img, pt1, pt2, (0,0,255), 3)
              dominance = line[0,0]
      pt1 = (line[0,0],line[0,1])
      pt2 = (line[0,2],line[0,3])
      cv2.line(img2, pt1, pt2, (0,0,255), 3)
      
  img_chopped = img[:,left_chop:right_chop,:]
  PERC = 30

  detector = ObjectDetection()
  detector.setModelTypeAsRetinaNet()
  detector.setModelPath("./temp_model.h5")
  detector.loadModel()

  custom_objects = detector.CustomObjects(person=False, car=False, bottle = True)
  main_obj, detections, extracted_obj = detector.detectCustomObjectsFromImage(input_image= img_chopped, 
                                                      input_type="array",
                                                      output_type = 'array', 
                                                      output_image_path= "./im.png", 
                                                      custom_objects=custom_objects,
                                                      extract_detected_objects=True,
                                                      minimum_percentage_probability=PERC)

  detections_all = detections.copy()
  img_chopped_black = img_chopped.copy()
  
  for i in range(len(detections)):
    img_chopped_black[detections[i]['box_points'][1]:detections[i]['box_points'][3],detections[i]['box_points'][0]:detections[i]['box_points'][2],:] = 0
  PERC = 5

  detector = ObjectDetection()
  detector.setModelTypeAsRetinaNet()
  detector.setModelPath("./temp_model.h5")
  detector.loadModel()

  custom_objects = detector.CustomObjects(person=False, car=False, bottle = True)
  main_obj, detections, extracted_obj = detector.detectCustomObjectsFromImage(input_image= img_chopped_black, 
                                                      input_type="array",
                                                      output_type = 'array', 
                                                      output_image_path= "./im.png", 
                                                      custom_objects=custom_objects,
                                                      extract_detected_objects=True,
                                                      minimum_percentage_probability=PERC)
  for i in detections:        
    detections_all.append(i)
  
  shelf_tuple = [(l['box_points'][0], l['box_points'][2]) for l in detections_all]
  shelf_tuple.append((0,0))
  shelf_tuple.append((img_chopped.shape[1], img_chopped.shape[1]))
  sorted_by_lower_bound = sorted(shelf_tuple, key=lambda tup: tup[0])
  merged = []

  for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            if higher[0] <= lower[1]:
                upper_bound = max(lower[1], higher[1])
                merged[-1] = (lower[0], upper_bound)  # replace by merged interval
            else:
                merged.append(higher)
  remove = []
  for i in range(len(merged)-1):
    if merged[i+1][0]-merged[i][1] > img_chopped.shape[1]*0.05:
      remove.append((merged[i][1], merged[i+1][0]))
      
  img_chopped_fin = img_chopped.copy()
  down_grade = 0
  for i in remove:
    #img_chopped_fin = img_chopped_fin[:,0:i[0] - down_grade + 1,:] + img_chopped_fin[:,i[1]- down_grade - 1:,:]
    img_chopped_fin = img_chopped_fin[:, np.r_[0:i[0] - down_grade + 1,i[1]- down_grade - 1: img_chopped_fin.shape[1]], :]
    down_grade += i[1] - i[0]

In [4]:
def colour_frame(img, frame, width=2, colour=[0, 0, 0]):
    """"""
    x1, y1, x2, y2 = frame['box_points']
    img[y1:y2, x1-width:x1+width] = colour
    img[y1:y2, x2-width:x2+width] = colour
    img[y1-width:y1+width, x1:x2] = colour
    img[y2-width:y2+width, x1:x2] = colour
    return img


def colour_map(shelf):
    MAP = {0: [255, 0, 0], 1: [0, 0, 255], 2: [0, 255, 0], 3: [0, 255, 255], 4: [255, 255, 0]}
    if shelf < 0:
        return [0, 0, 0 ]
    return MAP.get(shelf % len(MAP))


def compute_dominance_relations(detections):
    """"""
    for i, frame in enumerate(detections):
        x1, y1, x2, y2 = frame['box_points']
        for j, frame_ in enumerate(detections):
            x1_, y1_, x2_, y2_ = frame_['box_points']
            if y2 < y1_:
                dominated_by = frame_.setdefault('dominated_by', set())
                dominated_by.add(i)


def compute_seed_shelves(detections, polish=1/4):
    """"""
    frame_indeces = {i for i in range(len(detections))}    
    shelf = -1
    shelves = []
    while frame_indeces:
        shelf_frames = set()  
        for i in frame_indeces:
            frame = detections[i]
#            current_frame = colour_frame(main_obj, frame)
#            Image.fromarray(current_frame, 'RGB').show()
            dominated_by = (detections[i] for i in frame.get('dominated_by', {}))
            if all(dominating_frame.get('shelf', float('inf')) <= shelf 
                   for dominating_frame in dominated_by):
                x1, y1, x2, y2 = frame['box_points']
                upper_shelf_bottoms = [detections[i]['box_points'][3] for i in shelves[-1]] if shelf > -1 else []
                if sum(y2_ >= y2 - GAP for y2_ in upper_shelf_bottoms) > polish*len(upper_shelf_bottoms):
#                if all(y2_ >= y2 - GAP for y2_ in upper_shelf_bottoms):

                    frame['shelf'] = shelf
                    shelves[-1].add(i)
                else:
                    frame['shelf'] = shelf + 1
                    next_shelf = True
                    shelf_frames.add(i)
#                main_obj = colour_frame(main_obj, frame, colour=colour_map(shelf))
#                img = Image.fromarray(main_obj, 'RGB')
        frame_indeces -= shelf_frames
        if shelf > -1:
            frame_indeces -= shelves[-1]
        shelves.append(shelf_frames)

        shelf += 1 if next_shelf else 0
    return detections


def detect_nonshelves(detections, main_obj):
    """"""
    shelves = [set()  for _ in range(20)]
    for iframe, frame in enumerate(detections):
        shelves[frame['shelf']].add(iframe)
        
    image_width = main_obj.shape[1]
    to_remove = []
    for ishelf, shelf in enumerate(shelves[:-1]):
        lower_tops = [(detections[j]['box_points'][1], detections[j]['box_points'][2]) for j in shelves[ishelf+1]]
        if not lower_tops:
            continue
        noshelf = []
        for i in shelf:
            frame = detections[i]
            frame_bottom = frame['box_points'][3]
            frame_top = frame['box_points'][1]
            if sum(y1 < frame_bottom for y1, y2 in lower_tops) > 0*len(lower_tops)/2:
#                frame['shelf'] = -1
                noshelf.append(frame)
#        Image.fromarray(colour_shelves(noshelf, main_obj.copy()), 'RGB').show()
        safety_condition = noshelf and (min(f['box_points'][0] for f in noshelf) > image_width*1/2
                        or max(f['box_points'][2] for f in noshelf) < image_width*1/2)
        if safety_condition:
            for f in noshelf:
                f['shelf'] = -1
            to_remove.extend(noshelf)
    for frame in detections:
        if to_remove:
            if frame['box_points'][2] > image_width/2:
                if frame['box_points'][2] >= min(f['box_points'][0] for f in to_remove) > image_width/2:
                    frame['shelf'] = -1
            elif frame['box_points'][2] < image_width/2:
                if frame['box_points'][0] <= max(f['box_points'][2] for f in to_remove) < image_width/2:
                    frame['shelf'] = -1
    return detections


def detect_shelves(detections, polish=1/4):
    """Detekuje regaly na zaklade detekci lahvi. Vadi FALSE POSITIVES.
    V prvni iteraci je ale treba ponechat vysi FALSE POSITVES, abychom meli nizsi FALSE NEGATIVES,
    nebot FALSE NEGATIVES zase budou vadit nasledujicimu `correct` kroku.
    V pripade ze se v dalsich iteracich snizi FALSE POSITIVES, bude mozna treba zvysovat parametr `polish` smerem k 0.99.
    Parametr polish sleva "falesne regaly" do jednoho.
    
    Kazdy dict v `detections` dostane novy atribut `shelf` 
    """
    compute_dominance_relations(detections)   
    detections = compute_seed_shelves(detections, polish)
    return detections

def correct_shelves(detections, main_obj):
    """Odstrani lahve stojici v sousednich regalech. Vadi FALSE NEGATIVES. 
    
    Prvky v `detections` k odstraneni dostanou `shelf=-1`
    
    Input
    ----
    detections : list of dicts
        detections with `shelf` attribute from `detect` method
    main_obj : array 
        image from the bottle detector
    """
    detections = detect_nonshelves(detections, main_obj)
    return detections
        
def colour_shelves(detections, main_obj):
    """"""
    for frame in detections:
        if 'shelf' in frame:
            main_obj = colour_frame(main_obj, frame, colour=colour_map(frame['shelf']))
    return main_obj

In [ ]:
GAP = 15
detections = detect_shelves(detections_all) # zaradi do regalu
detections = correct_shelves(detections, img_chopped_fin)  # oznaci ty co nejsou v zadnem regalu
main_obj = colour_shelves(detections, img_chopped_fin.copy())
#img_chopped_fin
Img.fromarray(main_obj, 'RGB')

In [6]:
        shelf_tuple = list()
        for l in detections:
            if l['shelf'] == -1:
                shelf_tuple.append( (l['box_points'][0], l['box_points'][2]) )
        shelf_tuple.append((0,0))
        shelf_tuple.append((img_chopped_fin.shape[1], img_chopped_fin.shape[1]))
        sorted_by_lower_bound = sorted(shelf_tuple, key=lambda tup: tup[0])
        merged = []
        
        for higher in sorted_by_lower_bound:
                if not merged:
                    merged.append(higher)
                else:
                    lower = merged[-1]
                    if higher[0] <= lower[1]:
                        upper_bound = max(lower[1], higher[1])
                        merged[-1] = (lower[0], upper_bound)  # replace by merged interval
                    else:
                        merged.append(higher)
        #remove = []
        #for i in range(len(merged)-1):
        #    if merged[i+1][0]-merged[i][1] > img_chopped_fin.shape[1]*0.05:
        #      remove.append((merged[i][1], merged[i+1][0]))
        
        remove = merged
        img_chopped_completed = img_chopped_fin.copy()
        down_grade = 0
        for i in remove:
            #img_chopped_fin = img_chopped_fin[:,0:i[0] - down_grade + 1,:] + img_chopped_fin[:,i[1]- down_grade - 1:,:]
            img_chopped_completed = img_chopped_completed[:, np.r_[0:i[0] - down_grade + 1,i[1]- down_grade - 1: img_chopped_completed.shape[1]], :]
            down_grade += i[1] - i[0]

In [ ]:
Img.fromarray(img_chopped_completed, 'RGB')

In [18]:
cv2.imwrite("./test_write.jpg", img_chopped_completed)
!gsutil cp "./test_write.jpg" "gs://saleshousephotos/test.jpg"

True

In [ ]:
prediction_client = automl_v1beta1.PredictionServiceClient()
project_id = 'saleshouse-prototype'
model_id = 'ICN4760874677604180454'
name = 'projects/{}/locations/us-central1/models/{}'.format(project_id, model_id)

In [ ]:
tmp = [(0,0) for i in range(len(detections))]
class_tuples = {'RB': tmp, 'RBsf':tmp}
class_names = class_tuples.keys()
for e in range(len(extracted_obj)):
  cv2.imwrite("./img1.jpg", extracted_obj[e])
  with open("./img1.jpg", 'rb') as ff:
    content = ff.read()
  payload = {'image': {'image_bytes': content }}
  params = {}
  request = prediction_client.predict(name, payload, params)
  detections[e]['class'] = request.payload[0].display_name
  if detections[e]['class'] in class_names:
    class_tuples[detections[e]['class']][e] = (detections[e]['box_points'][0],detections[e]['box_points'][2])

In [ ]:
main_obj = colour_shelves_prediction(shelves, detections, main_obj)

In [ ]:
sz = main_obj.shape[1]
for clss in class_names:
  for k in range(len(shelves)):
    shelf_tuple = [class_tuples[clss][l] for l in shelves[k]]
    sorted_by_lower_bound = sorted(shelf_tuple, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
      if not merged:
          merged.append(higher)
      else:
          lower = merged[-1]
          if higher[0] <= lower[1]:
              upper_bound = max(lower[1], higher[1])
              merged[-1] = (lower[0], upper_bound)  # replace by merged interval
          else:
              merged.append(higher)
    zastoupeni = sum([i[1] - i[0] for i in merged])/sz 
    print(clss, k, zastoupeni)